In [2]:
import ollama

response = ollama.chat(
    model="llama3.2",
    messages=[
        {
            "role": "user",
            "content": "Tell me an interesting fact about elephants",
        },
    ],
)
print(response["message"]["content"])

Here's an interesting fact about elephants:

Elephants have a highly developed sense of empathy and can recognize and respond to the emotions of others, even in situations that are not directly related to themselves. They have been observed showing compassion and kindness towards each other, such as comforting a distressed family member or providing support to a dying companion.

In one famous study, researchers observed an elephant named Ruby at a wildlife sanctuary who was devastated when her favorite keeper, Amy, left the park after 24 years of caring for her. When Amy returned six months later, Ruby showed no recognition and displayed behaviors that indicated she had forgotten Amy's presence. However, when Amy started interacting with other elephants in the enclosure, including Ruby, the elephant began to display signs of distress again, suggesting that she was still deeply affected by Amy's departure.

This level of emotional intelligence and empathy is rare in the animal kingdom 

In [7]:
#Using RAG

import json
import os
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# Define files, questions, and debug flag
jsonl_path = "Data/r_nashville_comments.jsonl"  # Path to your JSONL file
chroma_db_path = "./chroma_db"
DEBUG = False

questions = ["What are the main issues with Nashville's transportation system?",
             "What is the most common complaint about Nashville's transportation system?"]

# Load the JSONL file and extract text content
def load_jsonl_data(file_path):
    docs = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            data = json.loads(line)
            text = data.get("text", "").strip()  # Extract the "text" field
            if text:
                docs.append(text)
    return docs

# Initialize embedding function using Ollama
embedding_model = OllamaEmbeddings(model="mxbai-embed-large")

# Load or create vector store
if os.path.exists(chroma_db_path):
    if DEBUG:
        print("Loading existing Chroma vector store...")
    vectorstore = Chroma(persist_directory=chroma_db_path, embedding_function=embedding_model)
else:
    if DEBUG:
        print("Creating new Chroma vector store...")
    raw_docs = load_jsonl_data(jsonl_path)

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.create_documents(raw_docs)

    # Store in Chroma vector database
    vectorstore = Chroma.from_documents(splits, embedding_model, persist_directory=chroma_db_path)
    vectorstore.persist()  # Save for future use

# Set up retriever
retriever = vectorstore.as_retriever()

# Load the local Llama3.2 model
llm = Ollama(model="llama3.2")

# Define custom system prompt
system_prompt = (
    "You are a traffic engineer. Your job is to obtain feedback from these Reddit posts "
    "and determine our city's weakest points in transportation infrastructure. "
    "Use the given context to identify issues and provide insightful analysis."
    "\n\n"
    "{context}"
)

# Create prompt template
rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create RAG pipeline
qa_chain = (
    {"context": retriever | (lambda docs: "\n\n".join(doc.page_content for doc in docs)), "input": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

# Interactive QA loop
for question in questions:
    print(f"Question: {question}")
    answer = qa_chain.invoke(question)
    print(f"\nAnswer: {answer}\n")


Loading existing Chroma vector store...
Question: What are the main issues with Nashville's transportation system?

Answer: As a traffic engineer, I've analyzed various Reddit posts and discussions about Nashville's transportation system. Based on the feedback, here are some key issues that stand out:

1. **Congestion and Bottlenecks**: Many users have reported congested roads, particularly on I-40, I-65, and Broadway. These areas tend to become gridlocked during rush hour, leading to frustration for commuters.
2. **Lack of Public Transportation Options**: Nashville's public transportation system is limited, with only a few bus routes that don't seem to cover the city effectively. Many users have suggested increasing the frequency and coverage of these services to alleviate traffic congestion.
3. **Insufficient Bike Infrastructure**: With Nashville's growing popularity as a cyclist-friendly city, users have noted that the bike lanes are scarce, and often poorly maintained. This lack of